# Challenge 1: Explore the data
## Task 3: Explore the data

In [41]:
import pandas as pd
import altair as alt
from dotenv import load_dotenv

In [42]:
load_dotenv()

True

In [2]:
df = pd.read_csv('truck_sales_data.csv')
df.head()

,transaction_id,at,truck_name,payment_method,total,truck_id,payment_method_id,fsa_rating,has_card_reader,truck_description
0,2795,2026-01-05 13:45:00,Cupcakes by Michelle,cash,299.0,3,1,5,True,"Handcrafted cupcakes made with high-quality, o..."
1,2793,2026-01-05 13:38:00,Cupcakes by Michelle,cash,299.0,3,1,5,True,"Handcrafted cupcakes made with high-quality, o..."
2,2789,2026-01-05 13:29:00,Cupcakes by Michelle,cash,799.0,3,1,5,True,"Handcrafted cupcakes made with high-quality, o..."
3,2787,2026-01-05 13:25:00,Cupcakes by Michelle,cash,299.0,3,1,5,True,"Handcrafted cupcakes made with high-quality, o..."
4,2783,2026-01-05 13:08:00,Cupcakes by Michelle,cash,299.0,3,1,5,True,"Handcrafted cupcakes made with high-quality, o..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5472 entries, 0 to 5471
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   transaction_id     5472 non-null   int64  
 1   at                 5472 non-null   object 
 2   truck_name         5472 non-null   object 
 3   payment_method     5472 non-null   object 
 4   total              5472 non-null   float64
 5   truck_id           5472 non-null   int64  
 6   payment_method_id  5472 non-null   int64  
 7   fsa_rating         5472 non-null   int64  
 8   has_card_reader    5472 non-null   bool   
 9   truck_description  5472 non-null   object 
dtypes: bool(1), float64(1), int64(4), object(4)
memory usage: 390.2+ KB


## Which truck has the highest number of transactions?

In [9]:
transactions_per_truck = (
    df.groupby('truck_name')['transaction_id']
    .count()
    .sort_values(ascending=False)
)

transactions_per_truck
print(f'The truc with the highest total number of transactions is, {transactions_per_truck.idxmax()} with: {transactions_per_truck.max()} transactions.')

The truc with the highest total number of transactions is, Burrito Madness with: 1416 transactions.


## Which truck has the lowest total transaction value?


In [12]:
total_value_per_truck = (
    df.groupby('truck_name')['total']
    .sum()
    .sort_values()

)

total_value_per_truck

print(f'The truck with the lowest total transaction value is, {total_value_per_truck.idxmin()} with a transaction value of: £{total_value_per_truck.min()}')

The truck with the lowest total transaction value is, Hartmann's Jellied Eels with a transaction value of: 90774.0


## What is the average transaction value?


In [16]:
average_transaction_value = df['total'].mean()

print(f'The average transaction value is: £{average_transaction_value:.2f}.')

The average transaction value is: £644.72.


## What is the average transaction value for each truck?


In [17]:
average_per_truck = (
    df.groupby('truck_name')['total']
    .mean()
    .sort_values(ascending=False)
)

average_per_truck

truck_name
Kings of Kebabs            807.625000
Burrito Madness            775.245763
SuperSmoothie              587.475177
Cupcakes by Michelle       583.785820
Yoghurt Heaven             541.584416
Hartmann's Jellied Eels    278.447853
Name: total, dtype: float64

## What proportion of transactions use cash?


In [21]:
payment_counts = df['payment_method'].value_counts(normalize=True)

cash_proportion = payment_counts.get("cash", 0)

print(f'The proportion of cash payments is: {(cash_proportion * 100):.2f}%')


The proportion of cash payments is: 54.11%


# Task 4: Visualise the data

In [24]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [25]:
df['at'] = pd.to_datetime(df['at'], errors='coerce')
df = df.dropna(subset=['at'])

## Average transaction total for each truck

In [29]:
avg_by_truck = (
    df.groupby('truck_name', as_index=False)['total']
    .mean()
    .rename(columns={'total': 'avg_transaction'})
)

bar_avg = (
    alt.Chart(avg_by_truck)
    .mark_bar()
    .encode(
        x=alt.Y('avg_transaction:Q', title='Average transaction value'),
        y=alt.X('truck_name:N', sort='-y', title='truck'),
        tooltip=[
            alt.Tooltip('truck_name:N', title='Truck'),
            alt.Tooltip('avg_transaction:Q', title='Average', format=',.2f')
        ]
    )
    .properties(title='Average transaction value by truck')
)

bar_avg

alt.Chart(...)

## Proportion of transactions made using cash or card

In [30]:
pay_counts = (
    df.groupby('payment_method', as_index=False)['transaction_id']
    .count()
    .rename(columns={'transaction_id': 'count'})

)

pie_payments = (
    alt.Chart(pay_counts)
    .mark_arc()
    .encode(
        theta=alt.Theta('count:Q', stack=True),
        color=alt.Color('payment_method:N', title='Payment method'),
        tooltip=[
            alt.Tooltip('payment_method:N', title='Method'),
            alt.Tooltip('count:Q', title='Transaction')
        ]
    )
    .properties(title='Proportion of transactions by payment method')
)

pie_payments

alt.Chart(...)

## Total transaction value each day

In [34]:
daily_totals = (
    df.assign(day=df['at'].dt.floor('D'))
        .groupby('day', as_index=False)['total']
        .sum()
        .rename(columns={'total': 'daily_total'})
)

line_daily = (
    alt.Chart(daily_totals)
    .mark_line()
    .encode(
        x=alt.X('day:T', title='Day'),
        y=alt.Y('daily_total:Q', title='Total transaction value'),
        tooltip=[
            alt.Tooltip('day:T', title='Day'),
            alt.Tooltip('daily_total:Q', title='Total', format=',.2f')
        ],
    )
    .properties(title='Total transaction value per day')
)

line_daily

alt.Chart(...)

# Challenge 4: Create and Query Tables
## Task 3: Query the Data

In [35]:
import boto3
import awswrangler as wr

In [36]:
database = 'c21-jordan-truck-data-lake'
table_transactions = 'transaction'
table_truck = 'truck'
table_payment_method = 'payment_method'

s3_output = 's3://c21-jordan-trucks/output/'
region = 'eu-west-2'

session = boto3.Session(region_name=region)

In [44]:
wr.catalog.get_tables(database=database, boto3_session=session)

<generator object get_tables at 0x1050a0600>

In [47]:
query = f"""
SELECT
t.transaction_id,
t.truck_id,
tr.truck_name,
t.payment_method_id,
pm.payment_method,
CAST(t.total AS DOUBLE) AS total,
t.at
FROM "{database}"."{table_transactions}" t
LEFT JOIN "{database}"."{table_truck}" tr
ON t.truck_id = tr.truck_id
LEFT JOIN "{database}"."{table_payment_method}" pm
ON t.payment_method_id = pm.payment_method_id
WHERE t.total IS NOT NULL
"""

df = wr.athena.read_sql_query(
    sql=query,
    database=database,
    s3_output=s3_output,
    boto3_session=session
)

df.head()

,transaction_id,truck_id,truck_name,payment_method_id,payment_method,total,at
0,1922,2,Kings of Kebabs,1,cash,700.0,1767898740000000000
1,1921,2,Kings of Kebabs,1,cash,850.0,1767898560000000000
2,1920,2,Kings of Kebabs,1,cash,700.0,1767897360000000000
3,1919,2,Kings of Kebabs,1,cash,700.0,1767896940000000000
4,1918,2,Kings of Kebabs,1,cash,850.0,1767896700000000000


In [48]:
df['total'] = pd.to_numeric(df['total'], errors='coerce')
df['at'] = pd.to_datetime(df['at'], errors='coerce')
df = df.dropna(subset=['total'])

### Which truck has the highest number of transactions?

In [50]:
transactions_count = (
    df.groupby("truck_name")["transaction_id"]
    .count()
    .sort_values(ascending=False)
)

highest_transaction_truck = transactions_count.idxmax()
highest_transaction_count = int(transactions_count.max())

print(f'Truck with the highest transactions is, {highest_transaction_truck} with {highest_transaction_count} transactions.')

Truck with the highest transactions is, Burrito Madness with 1416 transactions.


### Which truck has the lowest total transaction value?

In [52]:
total_by_truck = (
    df.groupby("truck_name")["total"]
    .sum()
    .sort_values(ascending=True)
)

lowest_revenue_truck = total_by_truck.idxmin()
lowest_revenue_value = float(total_by_truck.min())

print(f'The lowest revenue truck is {lowest_revenue_truck} with £{lowest_revenue_value:.2f}.')

The lowest revenue truck is Hartmann's Jellied Eels with £90774.00.


### What is the average transaction value?

In [57]:
avg_transaction_value = float(df["total"].mean())
print(f'£{avg_transaction_value:.2f}')

£644.72


### What is the average transaction value for each truck?

In [58]:
avg_by_truck = (
    df.groupby("truck_name")["total"]
    .mean()
    .sort_values(ascending=False)
)

avg_by_truck

truck_name
Kings of Kebabs            807.625000
Burrito Madness            775.245763
SuperSmoothie              587.475177
Cupcakes by Michelle       583.785820
Yoghurt Heaven             541.584416
Hartmann's Jellied Eels    278.447853
Name: total, dtype: float64

### What proportion of transactions use cash?


In [ ]:
payment_share = df["payment_method"].value_counts(normalize=True)
cash_proportion = float(payment_share.get("cash", 0.0))

cash_proportion, cash_proportion * 100

(0.5411184210526315, 54.11184210526315)

In [62]:
print(f"The proportion transactions that use cash are {cash_proportion*100:.2f}%")

The proportion transactions that use cash are 54.11%
